# Veri Kazıma 

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Tek Maç Kazıma

In [9]:
import pandas as pd
import requests
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# WebDriver kurulumu
driver = webdriver.Chrome()

# URL ve veri çekme
game_id = 12198159
url = f'https://api.sofascore.com/api/v1/event/{game_id}/statistics'
driver.get(url)

try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "pre")))
except Exception as e:
    print(f"Sayfa yüklenirken hata oluştu: {e}")
    driver.quit()

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
data = soup.get_text()

# JSON verisini okuma
all_shot_maps_data = pd.read_json(data)
all_shot_maps_data["game_id"] = game_id

# statistics sütununun içeriğini kontrol etme

# groups sütununu düzleştirme
def normalize_groups(df):
    normalized_rows = []
    
    for _, row in df.iterrows():
        game_id = row['game_id']
        stats = row['statistics']
        period = stats.get('period', 'UNKNOWN')
        groups = stats.get('groups', [])
        
        for group in groups:
            group_name = group.get('groupName', 'UNKNOWN')
            statistics = group.get('statisticsItems', [])
            
            for stat in statistics:
                stat.update({'game_id': game_id, 'period': period, 'groupName': group_name})
                normalized_rows.append(stat)
                
    return pd.DataFrame(normalized_rows)

# groups sütununu normalize etme
normalized_data = normalize_groups(all_shot_maps_data)



# API isteği ile takımların isimlerini alma
url = f'https://www.sofascore.com/api/v1/event/{game_id}'
response = requests.get(url)
data = response.json()

home_team = data['event']['homeTeam']['name']
away_team = data['event']['awayTeam']['name']

# Takım isimlerini yazdırma
print(f"Home Team: {home_team}")
print(f"Away Team: {away_team}")

# home ve away sütunlarını takım isimleri ile güncelleme
df = normalized_data[(normalized_data["period"] == "ALL")]

df.loc[df["name"] == "Expected goals", "home"] = home_team
df.loc[df["name"] == "Expected goals", "away"] = away_team

# Silmek istediğiniz sütunların listesi
columns_to_drop = ['compareCode', 'statisticsType', 'valueType','renderType','key','period','groupName','homeTotal','awayTotal']

# Bu sütunları df veri setinden silme
df = df.drop(columns=columns_to_drop)

# Güncellenmiş df veri setini kontrol etme
print(df.head())

# Son olarak, sadece "Expected goals" verilerini filtreleme
df = df[(df["name"] == "Expected goals")]

# Sonuçları yazdırma
print(df)


Home Team: Türkiye
Away Team: Georgia
               name     home     away  homeValue  awayValue   game_id
0   Ball possession      56%      44%      56.00      44.00  12198159
1    Expected goals  Türkiye  Georgia       3.01       1.37  12198159
2       Big chances        5        5       5.00       5.00  12198159
3       Total shots       22       14      22.00      14.00  12198159
4  Goalkeeper saves        3        5       3.00       5.00  12198159
             name     home     away  homeValue  awayValue   game_id
1  Expected goals  Türkiye  Georgia       3.01       1.37  12198159


In [11]:
df

,name,home,away,homeValue,awayValue,game_id
1,Expected goals,Türkiye,Georgia,3.01,1.37,12198159


# Birden çok maç kazıma

In [160]:
import pandas as pd
import requests
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# WebDriver kurulumu
driver = webdriver.Chrome()

# Game IDs
game_ids = [12528196, 12528217, 12528239,12745313,12528253,12528257,12528272,12528275,12530883,12530872,12530859]

# Boş bir DataFrame oluşturuluyor
all_normalized_data = pd.DataFrame()

for game_id in game_ids:
    # URL ve veri çekme
    url = f'https://api.sofascore.com/api/v1/event/{game_id}/statistics'
    driver.get(url)

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "pre")))
    except Exception as e:
        print(f"Sayfa yüklenirken hata oluştu: {e}")
        driver.quit()

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    data = soup.get_text()

    # JSON verisini okuma
    try:
        all_shot_maps_data = pd.read_json(data)
        all_shot_maps_data["game_id"] = game_id
    except ValueError:
        print(f"{game_id} için JSON verisi hatalı.")
        continue

    # groups sütununu düzleştirme
    def normalize_groups(df):
        normalized_rows = []
        
        for _, row in df.iterrows():
            game_id = row['game_id']
            stats = row['statistics']
            groups = stats.get('groups', [])
            
            # period bilgisi burada yer alabilir, kontrol ediyoruz
            period = stats.get('period', 'UNKNOWN')
            
            for group in groups:
                group_name = group.get('groupName', 'UNKNOWN')
                statistics = group.get('statisticsItems', [])
                
                for stat in statistics:
                    stat.update({'game_id': game_id, 'period': period, 'groupName': group_name})
                    normalized_rows.append(stat)
                    
        return pd.DataFrame(normalized_rows)

    # groups sütununu normalize etme
    normalized_data = normalize_groups(all_shot_maps_data)

    # API isteği ile takımların isimlerini alma
    url = f'https://www.sofascore.com/api/v1/event/{game_id}'
    response = requests.get(url)
    data = response.json()

    home_team = data['event']['homeTeam']['name']
    away_team = data['event']['awayTeam']['name']

    # Sadece takım isimlerini yazdırma
    print(f"{game_id} - Home Team: {home_team}")
    print(f"{game_id} - Away Team: {away_team}")

    # home ve away sütunlarını takım isimleri ile güncelleme
    normalized_data.loc[normalized_data["name"] == "Expected goals", "home"] = home_team
    normalized_data.loc[normalized_data["name"] == "Expected goals", "away"] = away_team

    # Silmek istediğiniz sütunların listesi
    columns_to_drop = ['compareCode', 'statisticsType', 'valueType','renderType','key','groupName','homeTotal','awayTotal']

    # Bu sütunları normalized_data veri setinden silme
    normalized_data = normalized_data.drop(columns=columns_to_drop)

    # "ALL" periyotlarını seçme
    if 'period' in normalized_data.columns:
        normalized_data = normalized_data[normalized_data["period"] == "ALL"]

    # Güncellenmiş veri setini all_normalized_data'ya ekleme
    all_normalized_data = pd.concat([all_normalized_data, normalized_data], ignore_index=True)

# WebDriver'ı kapatma
driver.quit()

# Son olarak, sadece "Expected goals" verilerini filtreleme
final_df = all_normalized_data[(all_normalized_data["name"] == "Expected goals")]

# Sonuçları yazdırma
print(final_df)


12528196 - Home Team: Kayserispor
12528196 - Away Team: Sivasspor
12528217 - Home Team: Konyaspor
12528217 - Away Team: Kayserispor
12528239 - Home Team: Kayserispor
12528239 - Away Team: Eyüpspor
12745313 - Home Team: Trabzonspor
12745313 - Away Team: Kayserispor
12528253 - Home Team: Göztepe
12528253 - Away Team: Kayserispor
12528257 - Home Team: Kayserispor
12528257 - Away Team: Beşiktaş
12528272 - Home Team: Başakşehir FK
12528272 - Away Team: Kayserispor
12528275 - Home Team: Kayserispor
12528275 - Away Team: Gaziantep FK
12530883 - Home Team: Hatayspor
12530883 - Away Team: Kayserispor
12530872 - Home Team: Kayserispor
12530872 - Away Team: Adana Demirspor
12530859 - Home Team: Kasımpaşa
12530859 - Away Team: Kayserispor
               name           home             away  homeValue  awayValue  \
1    Expected goals    Kayserispor        Sivasspor       1.83       1.18   
47   Expected goals      Konyaspor      Kayserispor       0.93       0.87   
89   Expected goals    Kayserisp

In [162]:
final_df

,name,home,away,homeValue,awayValue,game_id,period
1,Expected goals,Kayserispor,Sivasspor,1.83,1.18,12528196,ALL
47,Expected goals,Konyaspor,Kayserispor,0.93,0.87,12528217,ALL
89,Expected goals,Kayserispor,Eyüpspor,1.91,2.74,12528239,ALL
133,Expected goals,Trabzonspor,Kayserispor,2.71,2.40,12745313,ALL
177,Expected goals,Göztepe,Kayserispor,2.29,0.41,12528253,ALL
221,Expected goals,Kayserispor,Beşiktaş,0.52,2.15,12528257,ALL
264,Expected goals,Başakşehir FK,Kayserispor,1.95,1.30,12528272,ALL
308,Expected goals,Kayserispor,Gaziantep FK,1.83,1.49,12528275,ALL
351,Expected goals,Hatayspor,Kayserispor,1.74,1.71,12530883,ALL
393,Expected goals,Kayserispor,Adana Demirspor,1.16,0.24,12530872,ALL


In [164]:
# Filtering data for Fenerbahce's home and away matches
samsun_home = final_df[final_df['home'] == 'Kayserispor']
samsun_away = final_df[final_df['away'] == 'Kayserispor']

# Calculating the average xG for home and away matches
samsun_home_avg_xG = samsun_home['homeValue'].mean()
samsun_away_avg_xG = samsun_away['awayValue'].mean()

# Calculating overall average xG for Fenerbahce
samsun_overall_avg_xG = (samsun_home['homeValue'].sum() + samsun_away['awayValue'].sum()) / (
    len(samsun_home) + len(samsun_away))

samsun_home_avg_xG, samsun_away_avg_xG, samsun_overall_avg_xG


(1.45, 1.3150000000000002, 1.3763636363636365)